# ARCOS Data Exploration and Preprocessing (Part 1: Filtering)

This notebook explores the ARCOS (Automation of Reports and Consolidated Orders System) data and creates a filtered parquet file.

## Goals:
1. Examine the arcos_all.zip file contents
2. List and understand all columns
3. Filter by years (2006-2015) and states
4. Save filtered data to parquet

## Next Steps:
After completing this notebook, continue with `Preprocessing_Parquet_to_CountyYear.ipynb` to:
- Clean and validate the data
- Aggregate to county-year level
- Save final processed data

## Step 1: Import Required Libraries

In [13]:
import zipfile
import pandas as pd
import os

# Configure pandas display options for better table viewing
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)      # Show up to 100 rows
pd.set_option('display.width', None)        # Auto-detect width
pd.set_option('display.max_colwidth', None) # Show full column content
pd.set_option('display.float_format', '{:.2f}'.format)  # Format floats

print("Libraries imported successfully!")
print("Display options configured for Excel-like table view")

Libraries imported successfully!
Display options configured for Excel-like table view


## Step 2: Explore the ZIP File Contents

In [14]:
# Path to the zip file
zip_path = 'arcos_all.zip'

# List contents of the zip file
print("Contents of arcos_all.zip:")
print("=" * 60)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in file_list:
        file_info = zip_ref.getinfo(file)
        print(f"File: {file}")
        print(f"  Size: {file_info.file_size:,} bytes ({file_info.file_size / (1024**2):.2f} MB)")
        print(f"  Compressed Size: {file_info.compress_size:,} bytes")
        print("-" * 60)

print(f"\nTotal files in zip: {len(file_list)}")

Contents of arcos_all.zip:
File: arcos_all.tsv
  Size: 245,729,430,235 bytes (234345.85 MB)
  Compressed Size: 17,708,630,426 bytes
------------------------------------------------------------

Total files in zip: 1


## Step 3: Read Data from ZIP and Preview

In [15]:
# Read the data from the zip file (without extracting)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Get the first file (or you can specify the exact filename)
    data_file = file_list[0]
    
    print(f"Reading data from: {data_file}")
    print("=" * 60)
    
    # Read first few rows to understand structure
    with zip_ref.open(data_file) as f:
        df_preview = pd.read_csv(f, sep='\t', nrows=5)
        
print("\nFirst few rows:")
df_preview

Reading data from: arcos_all.tsv

First few rows:


,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,BUYER_DEA_NO,BUYER_BUS_ACT,BUYER_NAME,BUYER_ADDL_CO_INFO,BUYER_ADDRESS1,BUYER_ADDRESS2,BUYER_CITY,BUYER_STATE,BUYER_ZIP,BUYER_COUNTY,TRANSACTION_CODE,DRUG_CODE,NDC_NO,DRUG_NAME,Measure,MME_Conversion_Factor,Dosage_Strength,TRANSACTION_DATE,Combined_Labeler_Name,Reporter_family,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME
0,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,BT7499482,CHAIN PHARMACY,TARGET STORES A DIV.OF TARGET CORP.,TARGET STORE T-1364,ATTN: PHARMACY,895 SOUTH STATE ROAD 135,GREENWOOD,IN,46143,JOHNSON,S,9050,93015001,CODEINE,TAB,0.15,30.00,2011-01-14,Teva,McKesson Corporation,2.21,100.00,331.51
1,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,BT7499482,CHAIN PHARMACY,TARGET STORES A DIV.OF TARGET CORP.,TARGET STORE T-1364,ATTN: PHARMACY,895 SOUTH STATE ROAD 135,GREENWOOD,IN,46143,JOHNSON,S,9143,60951079770,OXYCODONE,TAB,1.50,10.00,2011-02-08,"Par Pharmaceutical, Inc.",McKesson Corporation,0.90,100.00,1344.75
2,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,BT7499482,CHAIN PHARMACY,TARGET STORES A DIV.OF TARGET CORP.,TARGET STORE T-1364,ATTN: PHARMACY,895 SOUTH STATE ROAD 135,GREENWOOD,IN,46143,JOHNSON,S,9193,406035901,HYDROCODONE,TAB,1.00,7.50,2011-03-07,SpecGx LLC,McKesson Corporation,0.45,100.00,454.05
3,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,BT7499482,CHAIN PHARMACY,TARGET STORES A DIV.OF TARGET CORP.,TARGET STORE T-1364,ATTN: PHARMACY,895 SOUTH STATE ROAD 135,GREENWOOD,IN,46143,JOHNSON,S,9250B,406577101,METHADONE,TAB,4.00,10.00,2011-03-01,SpecGx LLC,McKesson Corporation,3.58,400.00,14310.40
4,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,BT7499482,CHAIN PHARMACY,TARGET STORES A DIV.OF TARGET CORP.,TARGET STORE T-1364,ATTN: PHARMACY,895 SOUTH STATE ROAD 135,GREENWOOD,IN,46143,JOHNSON,S,9143,60951071270,OXYCODONE,TAB,1.50,10.00,2011-03-10,"Par Pharmaceutical, Inc.",McKesson Corporation,3.59,400.00,5379.00


## Step 4: Load Dataset and Examine Structure

In [16]:
# Load a sample of the dataset first for quick exploration
print("Loading dataset sample (100,000 rows)...")
print("=" * 60)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    with zip_ref.open(data_file) as f:
        df = pd.read_csv(f, sep='\t', nrows=100000)  # Load first 100k rows for faster exploration

print(f"Sample loaded successfully!")
print(f"\nDataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

print("\n" + "=" * 60)
print("DATASET INFO:")
print("=" * 60)
df.info()

print("\n" + "=" * 60)
print("Basic Statistics (Numeric Columns):")
print("=" * 60)
df.describe()

Loading dataset sample (100,000 rows)...


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_28556\2174128245.py:7: DtypeWarning: Columns (3,5,15,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep='\t', nrows=100000)  # Load first 100k rows for faster exploration


Sample loaded successfully!

Dataset Shape: 100,000 rows × 33 columns

DATASET INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   REPORTER_DEA_NO        100000 non-null  object 
 1   REPORTER_BUS_ACT       100000 non-null  object 
 2   REPORTER_NAME          100000 non-null  object 
 3   REPORTER_ADDL_CO_INFO  10875 non-null   object 
 4   REPORTER_ADDRESS1      100000 non-null  object 
 5   REPORTER_ADDRESS2      64874 non-null   object 
 6   REPORTER_CITY          100000 non-null  object 
 7   REPORTER_STATE         100000 non-null  object 
 8   REPORTER_ZIP           100000 non-null  int64  
 9   REPORTER_COUNTY        100000 non-null  object 
 10  BUYER_DEA_NO           100000 non-null  object 
 11  BUYER_BUS_ACT          100000 non-null  object 
 12  BUYER_NAME             100000 non-null  object 
 13  BUYER

,REPORTER_ZIP,BUYER_ZIP,MME_Conversion_Factor,Dosage_Strength,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME
count,100000.00,100000.00,100000.00,100000.00,100000.00,85861.00,100000.00
mean,52716.39,46532.47,19.95,15.68,165.59,2084.62,5150140.82
std,10680.49,522.87,38.23,27.33,26834.38,197365.18,807225041.15
min,7094.00,46001.00,0.10,0.00,0.00,0.01,0.00
25%,46241.00,46204.00,1.00,0.05,0.08,15.00,305.02
50%,47933.00,46241.00,1.00,7.50,0.54,100.00,1210.80
75%,60502.00,46815.00,3.00,10.00,1.88,200.00,3506.94
max,98001.00,47993.00,180.00,250.00,6489717.50,56499960.00,194691525000.00


## Step 5: Check for Missing Values

In [17]:
print("Missing Values Analysis:")
print("=" * 60)
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Missing_Percentage': missing_pct
})
missing_with_values = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_with_values) > 0:
    print(missing_with_values)
else:
    print("No missing values found!")

Missing Values Analysis:
                       Missing_Count  Missing_Percentage
REPORTER_ADDL_CO_INFO          89125               89.12
BUYER_ADDRESS2                 47110               47.11
BUYER_ADDL_CO_INFO             45677               45.68
REPORTER_ADDRESS2              35126               35.13
DOSAGE_UNIT                    14139               14.14
Combined_Labeler_Name            227                0.23
Measure                            4                0.00
BUYER_COUNTY                       1                0.00


## Step 6: Define Variables to Keep and State Filters

In [18]:
# Essential columns to keep
columns_to_keep = [
    'BUYER_STATE',              # State identifier
    'BUYER_COUNTY',             # County name
    'TRANSACTION_DATE',         # Extract year for temporal analysis
    'MME',                      # Pre-calculated Morphine Milligram Equivalents (primary outcome)
    'DOSAGE_UNIT',              # Total pills/dosage units (alternative measure)
    'DRUG_NAME',                # Specific opioid type (analyze drug-specific effects)
    'BUYER_BUS_ACT',            # Facility type (pharmacy, hospital, clinic)
    'Reporter_family',          # Distributor name (control for supply chain effects)
    'CALC_BASE_WT_IN_GM',       # Backup for MME verification
    'MME_Conversion_Factor'     # Backup for MME recalculation
]

# Treated states
treated_states = ['FL', 'WA']

# Control states for Florida
fl_primary_controls = ['GA', 'AL', 'SC']
fl_backup_controls = ['NC', 'TN', 'MS']

# Control states for Washington
wa_primary_controls = ['OR', 'CO', 'MN']
wa_backup_controls = ['NV', 'CA', 'VA']

# All states to keep (excluding Alaska)
states_to_keep = (
    treated_states + 
    fl_primary_controls + fl_backup_controls + 
    wa_primary_controls + wa_backup_controls
)

print("Columns to keep:")
for col in columns_to_keep:
    print(f"  - {col}")

print(f"\nStates to keep ({len(states_to_keep)} total):")
print(f"  Treated: {treated_states}")
print(f"  FL controls (primary): {fl_primary_controls}")
print(f"  FL controls (backup): {fl_backup_controls}")
print(f"  WA controls (primary): {wa_primary_controls}")
print(f"  WA controls (backup): {wa_backup_controls}")
print(f"\nExcluded: Alaska (AK) - county designation issues")

Columns to keep:
  - BUYER_STATE
  - BUYER_COUNTY
  - TRANSACTION_DATE
  - MME
  - DOSAGE_UNIT
  - DRUG_NAME
  - BUYER_BUS_ACT
  - Reporter_family
  - CALC_BASE_WT_IN_GM
  - MME_Conversion_Factor

States to keep (14 total):
  Treated: ['FL', 'WA']
  FL controls (primary): ['GA', 'AL', 'SC']
  FL controls (backup): ['NC', 'TN', 'MS']
  WA controls (primary): ['OR', 'CO', 'MN']
  WA controls (backup): ['NV', 'CA', 'VA']

Excluded: Alaska (AK) - county designation issues


## Step 7: Load and Filter Data (Save to Parquet)

In [ ]:
# Load data with selected columns only, filter years and states
print("Loading and filtering ARCOS data...")
print("=" * 60)
print("This will process the full dataset with filters applied...")
print("NOTE: ARCOS is a very large dataset (~500-900M rows), this may take 10-15 minutes")
print("")

# Read data in chunks and apply filters
chunks = []
chunk_size = 100000
total_rows_processed = 0
total_rows_kept = 0

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    with zip_ref.open(data_file) as f:
        reader = pd.read_csv(f, sep='\t', usecols=columns_to_keep, chunksize=chunk_size)
        
        for i, chunk in enumerate(reader):
            # Extract year from TRANSACTION_DATE
            chunk['year'] = pd.to_datetime(chunk['TRANSACTION_DATE']).dt.year
            
            # Filter for years 2006-2015
            chunk = chunk[(chunk['year'] >= 2006) & (chunk['year'] <= 2015)]
            
            # Filter for selected states (exclude Alaska)
            chunk = chunk[chunk['BUYER_STATE'].isin(states_to_keep)]
            
            total_rows_processed += chunk_size
            
            if not chunk.empty:
                total_rows_kept += len(chunk)
                chunks.append(chunk)
            
            # Progress indicator - show every 10M rows
            if (i + 1) % 100 == 0:
                keep_pct = (total_rows_kept / total_rows_processed) * 100 if total_rows_processed > 0 else 0
                print(f"  Processed: {total_rows_processed:,} rows | Kept: {total_rows_kept:,} ({keep_pct:.1f}%)")

# Combine all filtered chunks
df_filtered = pd.concat(chunks, ignore_index=True)

print(f"\n✓ Filtered data loaded!")
print(f"  Total rows processed: {total_rows_processed:,}")
print(f"  Total rows kept: {df_filtered.shape[0]:,} ({(df_filtered.shape[0]/total_rows_processed)*100:.1f}%)")
print(f"  Columns: {df_filtered.shape[1]}")

# Save filtered data to Parquet
filtered_file = 'arcos_filtered_2006_2015.parquet'
print(f"\nSaving filtered data to: {filtered_file}")
df_filtered.to_parquet(filtered_file, index=False, compression='snappy')

file_size = os.path.getsize(filtered_file)
print(f"✓ Saved to Parquet: {file_size:,} bytes ({file_size / (1024**2):.2f} MB)")

print("\n" + "=" * 60)
print("✓ PART 1 COMPLETE!")
print("=" * 60)
print("\nNext Step:")
print("  Open and run: Preprocessing_Parquet_to_CountyYear.ipynb")
print("  This notebook will:")
print("    - Load the filtered parquet file")
print("    - Perform data quality checks and cleaning")
print("    - Aggregate to county-year level")
print("    - Save final processed data")
print("\n" + "=" * 60)

Loading and filtering ARCOS data...
This will process the full dataset with filters applied...
NOTE: ARCOS is a very large dataset (~500-900M rows), this may take 10-15 minutes

  Processed: 10,000,000 rows | Kept: 0 (0.0%)
  Processed: 10,000,000 rows | Kept: 0 (0.0%)
  Processed: 20,000,000 rows | Kept: 440,201 (2.2%)
  Processed: 20,000,000 rows | Kept: 440,201 (2.2%)
  Processed: 30,000,000 rows | Kept: 1,495,527 (5.0%)
  Processed: 30,000,000 rows | Kept: 1,495,527 (5.0%)
  Processed: 40,000,000 rows | Kept: 2,729,169 (6.8%)
  Processed: 40,000,000 rows | Kept: 2,729,169 (6.8%)
  Processed: 50,000,000 rows | Kept: 3,468,522 (6.9%)
  Processed: 50,000,000 rows | Kept: 3,468,522 (6.9%)
  Processed: 60,000,000 rows | Kept: 4,393,388 (7.3%)
  Processed: 60,000,000 rows | Kept: 4,393,388 (7.3%)
  Processed: 70,000,000 rows | Kept: 6,170,339 (8.8%)
  Processed: 70,000,000 rows | Kept: 6,170,339 (8.8%)
  Processed: 80,000,000 rows | Kept: 8,017,027 (10.0%)
  Processed: 80,000,000 rows | 

## Step 8: Load from Parquet and Clean County Names

In [11]:
import os

# Check available CPU threads
num_threads = os.cpu_count()
print(f"System Information:")
print(f"  Total CPU threads available: {num_threads}")
print(f"  Recommendation: Use 25-50% of available threads for memory-intensive tasks")
print(f"  Suggested setting: {max(2, num_threads // 4)} threads")

System Information:
  Total CPU threads available: 22
  Recommendation: Use 25-50% of available threads for memory-intensive tasks
  Suggested setting: 5 threads


In [12]:
import polars as pl
import os

# Configure Polars to use fewer threads to avoid crashing
# With 22 threads available, using 6 threads (27%) for memory-intensive operations
os.environ["POLARS_MAX_THREADS"] = "6"

filtered_file = 'arcos_filtered_2006_2015.parquet'

print(f"Loading data from Parquet: {filtered_file}")
print("=" * 60)
print(f"Polars configured to use 6 threads (out of 22 available)")

df_arcos = pl.read_parquet(filtered_file)

print(f"\n✓ Data loaded from Parquet!")
print(f"  Rows: {df_arcos.shape[0]:,}")
print(f"  Columns: {df_arcos.shape[1]}")

print("\n" + "=" * 60)
print("Ready to proceed with data quality checks and cleaning...")
print("=" * 60)

Loading data from Parquet: arcos_filtered_2006_2015.parquet
Polars configured to use 6 threads (out of 22 available)

✓ Data loaded from Parquet!
  Rows: 218,477,461
  Columns: 11

Ready to proceed with data quality checks and cleaning...

✓ Data loaded from Parquet!
  Rows: 218,477,461
  Columns: 11

Ready to proceed with data quality checks and cleaning...


## Step 8a: Initial Data Quality Check

In [13]:
print("=" * 60)
print("STEP 8a: INITIAL DATA QUALITY CHECK")
print("=" * 60)

# 1. Check data shape
print(f"\n1. Data Shape:")
print(f"   Rows: {df_arcos.shape[0]:,}")
print(f"   Columns: {df_arcos.shape[1]}")

# 2. Check column names and types
print(f"\n2. Column Information:")
print(df_arcos.schema)

# 3. Check for null values
print(f"\n3. Null Values Check:")
null_counts = df_arcos.null_count()
print(null_counts)

# 4. Check year range
print(f"\n4. Year Range:")
year_stats = df_arcos.select([
    pl.col("year").min().alias("min_year"),
    pl.col("year").max().alias("max_year"),
    pl.col("year").n_unique().alias("unique_years")
])
print(year_stats)

# 5. Check states
print(f"\n5. States in Data:")
states = df_arcos.select("BUYER_STATE").unique().sort("BUYER_STATE")
print(f"   Unique states: {states.shape[0]}")
print(f"   States: {states.to_series().to_list()}")

# 6. Check for negative or zero values in key columns
print(f"\n6. Value Ranges Check:")
value_checks = df_arcos.select([
    (pl.col("MME") <= 0).sum().alias("MME_zero_or_neg"),
    (pl.col("DOSAGE_UNIT") <= 0).sum().alias("DOSAGE_UNIT_zero_or_neg"),
    pl.col("MME").min().alias("MME_min"),
    pl.col("MME").max().alias("MME_max"),
    pl.col("DOSAGE_UNIT").min().alias("DOSAGE_min"),
    pl.col("DOSAGE_UNIT").max().alias("DOSAGE_max")
])
print(value_checks)

print(f"\n{'=' * 60}")
print("✓ Initial quality check complete!")

STEP 8a: INITIAL DATA QUALITY CHECK

1. Data Shape:
   Rows: 218,477,461
   Columns: 11

2. Column Information:
Schema({'BUYER_BUS_ACT': String, 'BUYER_STATE': String, 'BUYER_COUNTY': String, 'DRUG_NAME': String, 'MME_Conversion_Factor': Float64, 'TRANSACTION_DATE': String, 'Reporter_family': String, 'CALC_BASE_WT_IN_GM': Float64, 'DOSAGE_UNIT': Float64, 'MME': Float64, 'year': Int32})

3. Null Values Check:
shape: (1, 11)
┌─────────────┬─────────────┬─────────────┬───────────┬───┬─────────────┬─────────────┬─────┬──────┐
│ BUYER_BUS_A ┆ BUYER_STATE ┆ BUYER_COUNT ┆ DRUG_NAME ┆ … ┆ CALC_BASE_W ┆ DOSAGE_UNIT ┆ MME ┆ year │
│ CT          ┆ ---         ┆ Y           ┆ ---       ┆   ┆ T_IN_GM     ┆ ---         ┆ --- ┆ ---  │
│ ---         ┆ u32         ┆ ---         ┆ u32       ┆   ┆ ---         ┆ u32         ┆ u32 ┆ u32  │
│ u32         ┆             ┆ u32         ┆           ┆   ┆ u32         ┆             ┆     ┆      │
╞═════════════╪═════════════╪═════════════╪═══════════╪═══╪═════════

## Step 8b: Clean County Names with Validation

In [14]:
print("=" * 60)
print("STEP 8b: CLEAN COUNTY NAMES")
print("=" * 60)

# Show sample of original county names
print("\n1. Sample of ORIGINAL county names (before cleaning):")
sample_counties_before = df_arcos.select("BUYER_COUNTY").unique().sort("BUYER_COUNTY").head(20)
print(sample_counties_before)

# Check for various issues
print("\n2. County Name Issues:")
county_checks = df_arcos.select([
    (pl.col("BUYER_COUNTY").str.contains("(?i)county")).sum().alias("has_county_suffix"),
    (pl.col("BUYER_COUNTY").str.strip_chars() != pl.col("BUYER_COUNTY")).sum().alias("has_whitespace"),
    pl.col("BUYER_COUNTY").n_unique().alias("unique_counties_before")
])
print(county_checks)

# Clean county names
print("\n3. Cleaning county names...")
df_arcos = df_arcos.with_columns([
    pl.col("BUYER_COUNTY")
    .str.strip_chars()  # Remove leading/trailing whitespace
    .str.to_uppercase()  # Convert to uppercase for consistency
    .str.replace(r"(?i)\s+COUNTY\s*$", "")  # Remove "COUNTY" suffix (case-insensitive)
    .str.strip_chars()  # Remove any trailing whitespace after removal
    .alias("BUYER_COUNTY")
])

print("✓ Cleaning applied!")

# Show sample of cleaned county names
print("\n4. Sample of CLEANED county names (after cleaning):")
sample_counties_after = df_arcos.select("BUYER_COUNTY").unique().sort("BUYER_COUNTY").head(20)
print(sample_counties_after)

# Verify cleaning results
print("\n5. Verification:")
county_checks_after = df_arcos.select([
    (pl.col("BUYER_COUNTY").str.contains("(?i)county")).sum().alias("still_has_county_suffix"),
    pl.col("BUYER_COUNTY").n_unique().alias("unique_counties_after")
])
print(county_checks_after)

print(f"\n{'=' * 60}")
print("✓ County name cleaning complete!")

STEP 8b: CLEAN COUNTY NAMES

1. Sample of ORIGINAL county names (before cleaning):
shape: (20, 1)
┌──────────────┐
│ BUYER_COUNTY │
│ ---          │
│ str          │
╞══════════════╡
│ null         │
│ ABBEVILLE    │
│ ACCOMACK     │
│ ADAMS        │
│ AIKEN        │
│ …            │
│ ALLENDALE    │
│ ALPINE       │
│ AMADOR       │
│ AMELIA       │
│ AMHERST      │
└──────────────┘

2. County Name Issues:
shape: (20, 1)
┌──────────────┐
│ BUYER_COUNTY │
│ ---          │
│ str          │
╞══════════════╡
│ null         │
│ ABBEVILLE    │
│ ACCOMACK     │
│ ADAMS        │
│ AIKEN        │
│ …            │
│ ALLENDALE    │
│ ALPINE       │
│ AMADOR       │
│ AMELIA       │
│ AMHERST      │
└──────────────┘

2. County Name Issues:
shape: (1, 3)
┌───────────────────┬────────────────┬────────────────────────┐
│ has_county_suffix ┆ has_whitespace ┆ unique_counties_before │
│ ---               ┆ ---            ┆ ---                    │
│ u32               ┆ u32            ┆ u32             

## Step 8c: Handle Invalid/Missing Values

In [15]:
print("=" * 60)
print("STEP 8c: HANDLE INVALID/MISSING VALUES")
print("=" * 60)

print(f"\n1. Initial row count: {df_arcos.shape[0]:,}")

# Check for null/missing values in critical columns
print("\n2. Checking for null values in critical columns:")
null_check = df_arcos.select([
    pl.col("BUYER_STATE").is_null().sum().alias("state_nulls"),
    pl.col("BUYER_COUNTY").is_null().sum().alias("county_nulls"),
    pl.col("year").is_null().sum().alias("year_nulls"),
    pl.col("MME").is_null().sum().alias("MME_nulls"),
    pl.col("DOSAGE_UNIT").is_null().sum().alias("DOSAGE_nulls")
])
print(null_check)

# Check for zero or negative values
print("\n3. Checking for invalid values (zero/negative):")
invalid_check = df_arcos.select([
    (pl.col("MME") <= 0).sum().alias("MME_invalid"),
    (pl.col("DOSAGE_UNIT") <= 0).sum().alias("DOSAGE_invalid"),
    ((pl.col("MME") <= 0) | (pl.col("DOSAGE_UNIT") <= 0)).sum().alias("either_invalid")
])
print(invalid_check)

# Filter out invalid records
print("\n4. Removing rows with invalid values...")
rows_before = df_arcos.shape[0]

df_arcos = df_arcos.filter(
    (pl.col("BUYER_STATE").is_not_null()) &
    (pl.col("BUYER_COUNTY").is_not_null()) &
    (pl.col("year").is_not_null()) &
    (pl.col("MME").is_not_null()) &
    (pl.col("DOSAGE_UNIT").is_not_null()) &
    (pl.col("MME") > 0) &
    (pl.col("DOSAGE_UNIT") > 0)
)

rows_after = df_arcos.shape[0]
rows_removed = rows_before - rows_after

print(f"   Rows before: {rows_before:,}")
print(f"   Rows after: {rows_after:,}")
print(f"   Rows removed: {rows_removed:,} ({(rows_removed/rows_before)*100:.2f}%)")

# Verify no invalid values remain
print("\n5. Verification - checking for remaining issues:")
verification = df_arcos.select([
    pl.col("BUYER_STATE").is_null().sum().alias("state_nulls"),
    pl.col("BUYER_COUNTY").is_null().sum().alias("county_nulls"),
    pl.col("year").is_null().sum().alias("year_nulls"),
    pl.col("MME").is_null().sum().alias("MME_nulls"),
    pl.col("DOSAGE_UNIT").is_null().sum().alias("DOSAGE_nulls"),
    (pl.col("MME") <= 0).sum().alias("MME_invalid"),
    (pl.col("DOSAGE_UNIT") <= 0).sum().alias("DOSAGE_invalid")
])
print(verification)

print(f"\n{'=' * 60}")
print("✓ Invalid values handled!")

STEP 8c: HANDLE INVALID/MISSING VALUES

1. Initial row count: 218,477,461

2. Checking for null values in critical columns:
shape: (1, 5)
┌─────────────┬──────────────┬────────────┬───────────┬──────────────┐
│ state_nulls ┆ county_nulls ┆ year_nulls ┆ MME_nulls ┆ DOSAGE_nulls │
│ ---         ┆ ---          ┆ ---        ┆ ---       ┆ ---          │
│ u32         ┆ u32          ┆ u32        ┆ u32       ┆ u32          │
╞═════════════╪══════════════╪════════════╪═══════════╪══════════════╡
│ 0           ┆ 6914         ┆ 0          ┆ 45        ┆ 27015222     │
└─────────────┴──────────────┴────────────┴───────────┴──────────────┘

3. Checking for invalid values (zero/negative):
shape: (1, 3)
┌─────────────┬────────────────┬────────────────┐
│ MME_invalid ┆ DOSAGE_invalid ┆ either_invalid │
│ ---         ┆ ---            ┆ ---            │
│ u32         ┆ u32            ┆ u32            │
╞═════════════╪════════════════╪════════════════╡
│ 50          ┆ 2484495        ┆ 2484542        │
└─

## Step 8d: Data Summary After Cleaning

In [16]:
print("=" * 60)
print("STEP 8d: CLEANED DATA SUMMARY")
print("=" * 60)

# Overall statistics
print("\n1. Data Dimensions:")
print(f"   Total rows: {df_arcos.shape[0]:,}")
print(f"   Total columns: {df_arcos.shape[1]}")

# Year distribution
print("\n2. Year Distribution:")
year_dist = df_arcos.group_by("year").agg(
    pl.count().alias("record_count")
).sort("year")
print(year_dist)

# State distribution
print("\n3. State Distribution:")
state_dist = df_arcos.group_by("BUYER_STATE").agg(
    pl.count().alias("record_count")
).sort("BUYER_STATE")
print(state_dist)

# County counts by state
print("\n4. County Counts by State:")
county_by_state = df_arcos.group_by("BUYER_STATE").agg(
    pl.col("BUYER_COUNTY").n_unique().alias("unique_counties")
).sort("BUYER_STATE")
print(county_by_state)

# Summary statistics for key variables
print("\n5. MME and Dosage Summary Statistics:")
summary_stats = df_arcos.select([
    pl.col("MME").sum().alias("total_MME"),
    pl.col("MME").mean().alias("avg_MME"),
    pl.col("MME").median().alias("median_MME"),
    pl.col("DOSAGE_UNIT").sum().alias("total_dosage"),
    pl.col("DOSAGE_UNIT").mean().alias("avg_dosage"),
    pl.col("DOSAGE_UNIT").median().alias("median_dosage")
])
print(summary_stats)

# Sample of clean data
print("\n6. Sample of Cleaned Data (first 10 rows):")
print(df_arcos.select(["BUYER_STATE", "BUYER_COUNTY", "year", "MME", "DOSAGE_UNIT"]).head(10))

print(f"\n{'=' * 60}")
print("✓ Data is clean and ready for aggregation!")

STEP 8d: CLEANED DATA SUMMARY

1. Data Dimensions:
   Total rows: 188,971,428
   Total columns: 11

2. Year Distribution:


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13172\56614353.py:13: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("record_count")


shape: (10, 2)
┌──────┬──────────────┐
│ year ┆ record_count │
│ ---  ┆ ---          │
│ i32  ┆ u32          │
╞══════╪══════════════╡
│ 2006 ┆ 15310663     │
│ 2007 ┆ 16500532     │
│ 2008 ┆ 17560995     │
│ 2009 ┆ 18390776     │
│ 2010 ┆ 19240405     │
│ 2011 ┆ 20536778     │
│ 2012 ┆ 20853779     │
│ 2013 ┆ 21365622     │
│ 2014 ┆ 20163325     │
│ 2015 ┆ 19048553     │
└──────┴──────────────┘

3. State Distribution:


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13172\56614353.py:20: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("record_count")


: 

## Step 9: Aggregate to County-Year Level

In [6]:
# Aggregate to county-year level
print("Aggregating to county-year level...")
print("=" * 60)

# Group by state, county, and year; sum MME and DOSAGE_UNIT using Polars
df_county_year = (
    df_arcos
    .group_by(["BUYER_STATE", "BUYER_COUNTY", "year"])
    .agg([
        pl.col("MME").sum().alias("opioid_shipments_mme"),
        pl.col("DOSAGE_UNIT").sum().alias("total_pills")
    ])
)

# Rename columns for clarity
df_county_year = df_county_year.rename({
    "BUYER_STATE": "state",
    "BUYER_COUNTY": "county_name"
})

print(f"✓ Aggregation complete!")
print(f"  Aggregated rows: {df_county_year.shape[0]:,}")
print(f"  Columns: {df_county_year.shape[1]}")

print("\n" + "=" * 60)
print("Sample of aggregated data:")
print(df_county_year.head(20))

Aggregating to county-year level...
✓ Aggregation complete!
  Aggregated rows: 10,309
  Columns: 5

Sample of aggregated data:
shape: (20, 5)
┌───────┬────────────────┬──────┬──────────────────────┬─────────────┐
│ state ┆ county_name    ┆ year ┆ opioid_shipments_mme ┆ total_pills │
│ ---   ┆ ---            ┆ ---  ┆ ---                  ┆ ---         │
│ str   ┆ str            ┆ i32  ┆ f64                  ┆ f64         │
╞═══════╪════════════════╪══════╪══════════════════════╪═════════════╡
│ FL    ┆ SUWANNEE       ┆ 2012 ┆ 3.6297e7             ┆ 1.9916e6    │
│ MN    ┆ WABASHA        ┆ 2013 ┆ 7.5659e6             ┆ 486771.008  │
│ MN    ┆ WATONWAN       ┆ 2013 ┆ 3.5791e6             ┆ 231739.0    │
│ NC    ┆ EDGECOMBE      ┆ 2012 ┆ 2.7625e7             ┆ 1.9263e6    │
│ NC    ┆ PENDER         ┆ 2014 ┆ 4.4222e7             ┆ 2.0916e6    │
│ …     ┆ …              ┆ …    ┆ …                    ┆ …           │
│ GA    ┆ PIKE           ┆ 2007 ┆ 3.8707e6             ┆ 234027.0    │
│ VA  

## Step 10: Validate Data Quality

In [9]:
# Validation checks
print("Data Quality Validation:")
print("=" * 60)

# Expected states (from Step 6)
expected_states = ['FL', 'WA', 'GA', 'AL', 'SC', 'NC', 'TN', 'MS', 'OR', 'CO', 'MN', 'NV', 'CA', 'VA']

# 1. Check for duplicates
duplicates = df_county_year.filter(pl.struct(["state", "county_name", "year"]).is_duplicated()).shape[0]
print(f"1. Duplicate rows: {duplicates}")
if duplicates > 0:
    print("   ⚠️  WARNING: Duplicates found!")
else:
    print("   ✓ No duplicates")

# 2. Verify year range
year_min = df_county_year["year"].min()
year_max = df_county_year["year"].max()
print(f"\n2. Year range: {year_min} - {year_max}")
if year_min == 2006 and year_max == 2015:
    print("   ✓ Correct year range (2006-2015)")
else:
    print(f"   ⚠️  Expected 2006-2015")

# 3. Confirm states present
unique_states = sorted(df_county_year["state"].unique().to_list())
print(f"\n3. States present ({len(unique_states)}):")
print(f"   {unique_states}")
missing_states = set(expected_states) - set(unique_states)
if missing_states:
    print(f"   ⚠️  Missing states: {missing_states}")
else:
    print(f"   ✓ All {len(expected_states)} expected states present")

# 4. Check for missing values
print(f"\n4. Missing values:")
null_counts = df_county_year.null_count()
total_nulls = sum(null_counts.row(0))
if total_nulls == 0:
    print("   ✓ No missing values")
else:
    print(null_counts)

# 5. Summary statistics
print(f"\n5. Summary statistics:")
print(f"   Total counties: {df_county_year['county_name'].n_unique()}")
print(f"   Total state-county-year observations: {df_county_year.shape[0]:,}")
print(f"   Years per county (avg): {df_county_year.shape[0] / df_county_year['county_name'].n_unique():.1f}")

print("\n" + "=" * 60)
print("Validation complete!")

Data Quality Validation:
1. Duplicate rows: 0
   ✓ No duplicates

2. Year range: 2006 - 2015
   ✓ Correct year range (2006-2015)

3. States present (14):
   ['AL', 'CA', 'CO', 'FL', 'GA', 'MN', 'MS', 'NC', 'NV', 'OR', 'SC', 'TN', 'VA', 'WA']
   ✓ All 14 expected states present

4. Missing values:
shape: (1, 5)
┌───────┬─────────────┬──────┬──────────────────────┬─────────────┐
│ state ┆ county_name ┆ year ┆ opioid_shipments_mme ┆ total_pills │
│ ---   ┆ ---         ┆ ---  ┆ ---                  ┆ ---         │
│ u32   ┆ u32         ┆ u32  ┆ u32                  ┆ u32         │
╞═══════╪═════════════╪══════╪══════════════════════╪═════════════╡
│ 0     ┆ 16          ┆ 0    ┆ 0                    ┆ 0           │
└───────┴─────────────┴──────┴──────────────────────┴─────────────┘

5. Summary statistics:
   Total counties: 778
   Total state-county-year observations: 10,309
   Years per county (avg): 13.3

Validation complete!


## Step 11: Save Final Processed Data to Parquet

In [10]:
# Create output directory if it doesn't exist
output_dir = 'data/intermediate'
os.makedirs(output_dir, exist_ok=True)

# Save to parquet
output_file = os.path.join(output_dir, 'arcos_county_year.parquet')

print(f"Saving final processed data to: {output_file}")
print("=" * 60)

df_county_year.write_parquet(output_file, compression='snappy')

# Check file size
file_size = os.path.getsize(output_file)
print(f"✓ File saved successfully!")
print(f"  File: {output_file}")
print(f"  Size: {file_size:,} bytes ({file_size / 1024:.2f} KB)")
print(f"  Rows: {df_county_year.shape[0]:,}")
print(f"  Columns: {df_county_year.shape[1]}")

print("\n" + "=" * 60)
print("Final Dataset Summary:")
print("=" * 60)
print(f"  Time period: {df_county_year['year'].min()} - {df_county_year['year'].max()}")
print(f"  States: {df_county_year['state'].n_unique()}")
print(f"  Counties: {df_county_year['county_name'].n_unique()}")
print(f"  Total observations: {df_county_year.shape[0]:,}")
print(f"\n  Columns:")
for col in df_county_year.columns:
    print(f"    - {col}")

print("\n✓ Preprocessing complete! Data ready for analysis.")

Saving final processed data to: data/intermediate\arcos_county_year.parquet
✓ File saved successfully!
  File: data/intermediate\arcos_county_year.parquet
  Size: 187,215 bytes (182.83 KB)
  Rows: 10,309
  Columns: 5

Final Dataset Summary:
  Time period: 2006 - 2015
  States: 14
  Counties: 778
  Total observations: 10,309

  Columns:
    - state
    - county_name
    - year
    - opioid_shipments_mme
    - total_pills

✓ Preprocessing complete! Data ready for analysis.
